In [1]:
from datetime import datetime, timedelta
# import cartopy.crs as ccrs
import os
import s2sphere as s2
import json
import pandas as pd
import plotly
import plotly.graph_objects as go
from tqdm.notebook import tqdm
# os.environ["CARTOPY_USER_BACKGROUNDS"] = "./../BG"

In [2]:
def get_s2cell(lat, long, level):
    generator_obj = s2.CellId().from_lat_lng(s2.LatLng.from_degrees(lat, long)).parent(level)
    token = generator_obj.to_token()
    lat = generator_obj.to_lat_lng().lat().degrees
    lng = generator_obj.to_lat_lng().lng().degrees
    return (token, lat, lng)
get_s2cells = lambda df: pd.DataFrame(df.apply(lambda row: get_s2cell(row['lat_x'], row['lon_x'], 14), axis=1).tolist(), columns=['token', 'slat', 'slng'])


In [3]:
all_tweets = pd.read_pickle("../data/cleanmoji.pkl")#.head(10000)
all_tweets['created_at_date'] = pd.to_datetime(all_tweets['created_at'], infer_datetime_format=True) 
all_tweets = all_tweets.merge(get_s2cells(all_tweets), left_index=True, right_index=True)
all_tweets.set_index('created_at_date', inplace=True)
plot = all_tweets.resample('1H').mean()[['slat', 'slng', 'Sent_1']]
track = pd.read_pickle("../data/track.pkl")
track['TIME_'] = track['TIME_'].dt.tz_localize("UTC")
us_tweets = all_tweets[all_tweets['cc'] == "US"]
all_tweets['cc'].unique(), all_tweets.columns


(array(['US', 'BS', 'BR', 'DO', 'AR', 'AU', 'PR', 'NL', 'MX', 'GB', 'DE',
        'CL', 'TH', 'HT', 'VI', 'GH', 'KY', 'ES', 'GN', 'GR', 'JM', 'IN',
        'CA', 'FR', 'PH', 'SE', 'HN', 'PA', 'PE', 'CO', 'CR', 'ZA', 'IE',
        'ID', 'BD', 'JP', 'NG', 'MY', 'GP', 'CH', 'VE', 'SA', 'BE', 'PT',
        'CN', 'BB', 'LK', 'LB', 'DK', 'AT', 'VN', 'IT'], dtype=object),
 Index(['full_text', 'created_at', 'id', 'lon_x', 'lat_x', 'lat_y', 'lon_y',
        'name', 'admin1', 'admin2', 'cc', 'Text', 'Top5%', 'Emoji_1', 'Emoji_2',
        'Emoji_3', 'Emoji_4', 'Emoji_5', 'Pct_1', 'Pct_2', 'Pct_3', 'Pct_4',
        'Pct_5', 'Sent_1', 'Sent_2', 'Sent_3', 'Sent_4', 'Sent_5', 'token',
        'slat', 'slng'],
       dtype='object'))

In [4]:
def viz(tweets, counts, cur_track, idx, time):
    fig = go.Figure(data=go.Scattergeo(
        lat = tweets['lat_x'],
        lon = tweets['lon_x'],
        text = tweets['Emoji_1'],
        mode = 'text',
        marker = dict(
            opacity = 0.7,
            size = tweets['Emoji_1'].map(counts),
            colorbar = dict(
                titleside = "right",
    #             outlinecolor = "rgba(68, 68, 68, 0)",
                ticks = "outside",
                showticksuffix = "last",
                dtick = 0.1
            )
        ),
        textfont = dict(
        size = (tweets['Emoji_1'].map(counts))
        )
    ))
    fig.add_trace(go.Scattergeo(
        lat = cur_track['LAT'],
        lon = cur_track['LON'],
        mode = 'markers',
        marker = dict(
            opacity = 0.25,
            size = cur_track['INTENSITY'] - 20
        ))
    )
    
    fig.update_layout(
        width=1920,
        height=1080,
        geo = dict(
            scope = 'world',
            showland = True,
            showocean = True,
            landcolor = "rgb(212, 212, 212)",
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(46, 74, 158)",
            oceancolor = "rgb(46, 74, 158)",
            showsubunits = True,
            showcountries = True,
            resolution = 50,
            projection = dict(
                type = 'conic conformal',
                rotation_lon = -80
            ),
            lonaxis = dict(
                showgrid = True,
                gridwidth = 0.5,
                range= [ -120.0, -15.0 ],
                dtick = 5
            ),
            lataxis = dict (
                showgrid = True,
                gridwidth = 0.5,
                range= [ 15.0, 40.0 ],
                dtick = 5
            )
        ),
        title=dict(
                text = f"Hurricane Sandy @ {time.strftime('%d %b %I:%M %p')}",
                y = 0.9,
                x = 0.5,
                xanchor ='center',
                yanchor ='top'
            ),
        font=dict(
                family="Open Sans",
                size=18,
                color="Grey"
            )
    
    )

    fig.write_image(f"../images/fig{str(idx).zfill(4)}.svg")
    
# viz(tweets, counts, cur_track, 1, d)

In [7]:
for i, d in (enumerate(plot.index)):
    if i == len(plot.index) - 1:
        break
    n = plot.index[i+1] 
    tweets = all_tweets[(all_tweets.index < n) & (d <= all_tweets.index)]
    cur_track = track[track['TIME_'] < n]
    counts = pd.melt(tweets[[f'Emoji_{i+1}' for i in range(5)]])['value'].value_counts()
    count = tweets.shape[0]
    
    maxmoji = tweets[[f'Emoji_{i+1}' for i in range(5)]].values.flatten().max() if count > 0 else '❌'
#     print(d.strftime("%d %b %H:%M %p"), maxmoji, count)
    viz(tweets, counts, cur_track, i, d)



In [30]:
!cd .. && ffmpeg -y -framerate 2  -pattern_type glob -i "./images/*.svg" -c:v libx264 -r 30 -pix_fmt yuv420p sandy_scene.mp4


ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with Apple LLVM version 10.0.0 (clang-1000.11.45.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.3.1_4 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --disable-libjack --disable-indev=jack
  libavutil      56. 51.100 / 

In [136]:
# color_dict = {'TROPICAL DEPRESSION':'#AEF100', 'TROPICAL STORM':'#FFD600', 'HURRICANE-1':'#FF6440', 'HURRICANE-2':'#8506A9'}
# cur_track.head()

,OBJECTID,STORMNAME,DTG,YEAR_,MONTH_,DAY_,HHMM,TAU,MSLP,BASIN,STORMNUM,STORMTYPE,INTENSITY,SS,LAT,LON,TIME_,X,Y,TIME_2
0,1,INVEST,2012102118,2012,10,21,1800,0,1006,AL,18,Low,25,0,14.3,-77.4,2012-10-21 18:00:00-04:00,-8.616129e+06,1.608657e+06,2012/10/21 18:00:00+00
1,2,INVEST,2012102200,2012,10,22,0,0,1005,AL,18,Low,25,0,13.9,-77.8,2012-10-22 00:00:00-04:00,-8.660657e+06,1.562746e+06,2012/10/22 00:00:00+00
2,3,INVEST,2012102206,2012,10,22,600,0,1003,AL,18,Low,25,0,13.5,-78.2,2012-10-22 06:00:00-04:00,-8.705184e+06,1.516914e+06,2012/10/22 06:00:00+00
3,4,EIGHTEEN,2012102212,2012,10,22,1200,0,1002,AL,18,Tropical Depression,30,0,13.1,-78.6,2012-10-22 12:00:00-04:00,-8.749712e+06,1.471159e+06,2012/10/22 12:00:00+00
4,5,EIGHTEEN,2012102218,2012,10,22,1800,0,1000,AL,18,Tropical Storm,35,0,12.7,-78.7,2012-10-22 18:00:00-04:00,-8.760844e+06,1.425478e+06,2012/10/22 18:00:00+00


## Sandbox

In [42]:
us_tweets.groupby(['Emoji_1'])['full_text'].count().reset_index().sort_values('full_text')

,Emoji_1,full_text
17,💕,5
18,💖,24
30,😌,31
59,🙊,49
50,😪,50
...,...,...
45,😡,3972
61,🙏,4524
54,😳,5332
11,👍,6745


In [162]:
# ny_tweets = all_tweets[(all_tweets['admin1'] == 'New Jersey') | (all_tweets['admin1'] == 'New York')]
# for i, d in enumerate(plot.index):
#     if i == len(plot.index) - 1:
#         break
#     n = plot.index[i+1] 
#     tweets = ny_tweets[(ny_tweets.index < n) & (d <= ny_tweets.index)]
#     ntweets = tweets.shape[0]
#     maxmoji = tweets[[f'Emoji_{i+1}' for i in range(1)]].values.flatten().max() if ntweets > 0 else '❌'
#     count = tweets[[f'Emoji_{i+1}' for i in range(1)]].values.flatten().argmax() if ntweets > 0 else 0
#     perc = round(100 * count / ntweets, 3)if ntweets > 0 else 0
#     print(d.strftime("%d %b %H:%M %p"), maxmoji, perc)
# #     ax.scatter(tweets['slng'], tweets['slat'], s=1, alpha=0.8, transform=ccrs.PlateCarree())


In [19]:
a = {'❌': 10, '❌': 23, '❌': 13, '❌': 10, '❌': 13}

In [2]:
# Load Apple Color Emoji font
prop = FontProperties(fname='/usr/share/fonts/truetype/noto/NotoColorEmoji.ttf')

# Load Apple Color Emoji font
# prop = FontProperties(fname='/System/Library/Fonts/Apple Color Emoji.ttc')

#sample arrays
104:106: syntax error: A unknown token can’t go after this “"”. (-2740)y_array = np.array([1, 2, 3, 4])
emoji_array = ['😂', '😃', '😛', '😸']

def emoji_scatter(x_array, y_array, emoji_array, savename = None):
    #set up the plot
    fig, ax = plt.subplots()
    ax.scatter(x_array, y_array, color="white")

    #annotate with your emojis
    for i, txt in enumerate(emoji_array):
        ax.annotate(txt, (x_array[i], y_array[i]),
        ha="center",
        va="bottom",
        fontsize=30,
        fontproperties=prop)

    if savename:
        fig.savefig(savename)

    plt.show()

emoji_scatter(x_array, y_array, emoji_array, 'emoji_scatterplot')